In [1]:
import requests
from bs4 import BeautifulSoup
from operator import itemgetter, attrgetter
from tqdm.notebook import tqdm
from multiprocessing.dummy import Pool as ThreadPool, Queue
import gzip, codecs
from itertools import chain
from time import sleep
from fake_useragent import UserAgent
from functools import partial
from contextlib import ExitStack
import re
from collections import OrderedDict
import os, sys
import json
import numpy as np
import pandas as pd
import shutil
pd.set_option('display.max_columns',50)

In [2]:
DOMAIN = 'https://shop.relod.ru'

In [3]:
ua = UserAgent()
UA = ua.firefox
UA

'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0'

In [4]:
def request_page(url, method, n_attempts=5, t_sleep=1, **kwargs):
    """
        Проверяет, что страница скачана (если страница не скачана, то пробует ее перекачать)
    """ 
    while n_attempts > 0:
        page = requests.request(method, url, **kwargs)
        if page.ok:
            return page
        sleep(t_sleep)
        n_attempts -= 1
get_page = partial(request_page, method='GET')
post_page = partial(request_page, method='POST')

In [5]:
data = {
    "siteId":	"s1",
    "template":	".default.5066805247fd90f4681c9458759019c47aeeee73c2035d99874d41cd9e7eb382",
    "parameters": "YToxNzM6e3M6MTg6IkNPTVBPTkVOVF9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxMToiSUJMT0NLX1RZUEUiO3M6NzoiY2F0YWxvZyI7czo5OiJJQkxPQ0tfSUQiO3M6MToiMiI7czoxMDoiU0VDVElPTl9JRCI7czo0OiI0NTc3IjtzOjEyOiJTRUNUSU9OX0NPREUiO047czoxMToiRklMVEVSX05BTUUiO3M6MTY6IkNBVEFMT0dfUFJPRFVDVFMiO3M6MTk6IklOQ0xVREVfU1VCU0VDVElPTlMiO3M6MToiQSI7czoxOToiU0hPV19BTExfV09fU0VDVElPTiI7czoxOiJOIjtzOjEzOiJDVVNUT01fRklMVEVSIjtzOjA6IiI7czoxODoiSElERV9OT1RfQVZBSUxBQkxFIjtzOjE6Ik4iO3M6MjU6IkhJREVfTk9UX0FWQUlMQUJMRV9PRkZFUlMiO3M6MToiWSI7czoxODoiRUxFTUVOVF9TT1JUX0ZJRUxEIjtzOjE1OiJQUk9QRVJUWV9SQVRJTkciO3M6MTg6IkVMRU1FTlRfU09SVF9PUkRFUiI7czo0OiJkZXNjIjtzOjE5OiJFTEVNRU5UX1NPUlRfRklFTEQyIjtzOjQ6Im5hbWUiO3M6MTk6IkVMRU1FTlRfU09SVF9PUkRFUjIiO3M6NDoiZGVzYyI7czoxNzoiT0ZGRVJTX1NPUlRfRklFTEQiO3M6MjoiaWQiO3M6MTc6Ik9GRkVSU19TT1JUX09SREVSIjtzOjM6ImFzYyI7czoxODoiT0ZGRVJTX1NPUlRfRklFTEQyIjtzOjU6InNob3dzIjtzOjE4OiJPRkZFUlNfU09SVF9PUkRFUjIiO3M6MzoiYXNjIjtzOjE4OiJQQUdFX0VMRU1FTlRfQ09VTlQiO3M6MjoiMjAiO3M6MTg6IkxJTkVfRUxFTUVOVF9DT1VOVCI7czoxOiI0IjtzOjIwOiJQUk9QRVJUWV9DT0RFX01PQklMRSI7YToyOntpOjA7czo4OiJBVFJfSVNCTiI7aToxO3M6MTM6IkFUUl9QVUJMSVNIRVIiO31zOjE3OiJPRkZFUlNfRklFTERfQ09ERSI7YTo2OntpOjA7czo0OiJOQU1FIjtpOjE7czoxMjoiUFJFVklFV19URVhUIjtpOjI7czoxNToiUFJFVklFV19QSUNUVVJFIjtpOjM7czoxMToiREVUQUlMX1RFWFQiO2k6NDtzOjE0OiJERVRBSUxfUElDVFVSRSI7aTo1O3M6MDoiIjt9czoyMDoiT0ZGRVJTX1BST1BFUlRZX0NPREUiO2E6Mjp7aTowO3M6NjoiTk9USUNFIjtpOjE7czowOiIiO31zOjEyOiJPRkZFUlNfTElNSVQiO3M6MToiMCI7czoyMDoiUFJPRFVDVF9ESVNQTEFZX01PREUiO3M6MToiWSI7czoxMzoiQUREX1BJQ1RfUFJPUCI7czoxNzoiQVRSX0VYVFJBX1BJQ1RVUkUiO3M6MTk6Ik9GRkVSX0FERF9QSUNUX1BST1AiO3M6MToiLSI7czoxNjoiT0ZGRVJfVFJFRV9QUk9QUyI7YToxOntpOjA7czo2OiJOT1RJQ0UiO31zOjIwOiJQUk9EVUNUX1NVQlNDUklQVElPTiI7czoxOiJZIjtzOjIxOiJTSE9XX0RJU0NPVU5UX1BFUkNFTlQiO3M6MToiWSI7czoxNDoiU0hPV19PTERfUFJJQ0UiO3M6MToiWSI7czoxNzoiU0hPV19NQVhfUVVBTlRJVFkiO3M6MToiQSI7czoyMToiQlhSX1NIT1dfTUFYX1FVQU5USVRZIjtzOjE6IkEiO3M6MTY6IlNIT1dfQ0xPU0VfUE9QVVAiO3M6MToiWSI7czoxMjoiTUVTU19CVE5fQlVZIjtzOjEyOiLQmtGD0L/QuNGC0YwiO3M6MjI6Ik1FU1NfQlROX0FERF9UT19CQVNLRVQiO3M6MTc6ItCSINC60L7RgNC30LjQvdGDIjtzOjE4OiJNRVNTX0JUTl9TVUJTQ1JJQkUiO3M6NDQ6ItCj0LLQtdC00L7QvNC40YLRjCDQviDQv9C+0YHRgtGD0L/Qu9C10L3QuNC4IjtzOjE1OiJNRVNTX0JUTl9ERVRBSUwiO3M6MzQ6ItCf0L7QtNGA0L7QsdC90LXQtSDQviDRgtC+0LLQsNGA0LUiO3M6MTg6Ik1FU1NfTk9UX0FWQUlMQUJMRSI7czoyNDoi0J3QtdGCINCyINC90LDQu9C40YfQuNC4IjtzOjExOiJTRUNUSU9OX1VSTCI7czozMToiL2NhdGFsb2ctcHJvZHVjdHMvI1NFQ1RJT05fSUQjLyI7czoxMDoiREVUQUlMX1VSTCI7czozMzoiL2NhdGFsb2ctcHJvZHVjdHMvI0VMRU1FTlRfQ09ERSMvIjtzOjE5OiJTRUNUSU9OX0lEX1ZBUklBQkxFIjtzOjEwOiJTRUNUSU9OX0lEIjtzOjEwOiJDQUNIRV9UWVBFIjtzOjE6IkEiO3M6MTA6IkNBQ0hFX1RJTUUiO3M6ODoiMzYwMDAwMDAiO3M6MTI6IkNBQ0hFX0ZJTFRFUiI7czoxOiJZIjtzOjEyOiJDQUNIRV9HUk9VUFMiO3M6MToiWSI7czo5OiJTRVRfVElUTEUiO3M6MToiWSI7czoxMzoiQlJPV1NFUl9USVRMRSI7czoxOiItIjtzOjEzOiJNRVRBX0tFWVdPUkRTIjtzOjE6Ii0iO3M6MTY6Ik1FVEFfREVTQ1JJUFRJT04iO3M6MToiLSI7czoxNzoiU0VUX0xBU1RfTU9ESUZJRUQiO3M6MToiWSI7czoyNDoiVVNFX01BSU5fRUxFTUVOVF9TRUNUSU9OIjtzOjE6IlkiO3M6MTg6IkFERF9TRUNUSU9OU19DSEFJTiI7czoxOiJOIjtzOjE1OiJBQ1RJT05fVkFSSUFCTEUiO3M6NjoiYWN0aW9uIjtzOjE5OiJQUk9EVUNUX0lEX1ZBUklBQkxFIjtzOjI6ImlkIjtzOjEwOiJQUklDRV9DT0RFIjthOjE6e2k6MDtzOjEwOiJCQVNFX1BSSUNFIjt9czoxNToiVVNFX1BSSUNFX0NPVU5UIjtzOjE6Ik4iO3M6MTY6IlNIT1dfUFJJQ0VfQ09VTlQiO3M6MDoiIjtzOjE3OiJQUklDRV9WQVRfSU5DTFVERSI7czoxOiJZIjtzOjE2OiJDT05WRVJUX0NVUlJFTkNZIjtzOjE6IlkiO3M6MTE6IkNVUlJFTkNZX0lEIjtzOjM6IlJVQiI7czoxMDoiQkFTS0VUX1VSTCI7czoyMDoiL3BlcnNvbmFsL2Jhc2tldC5waHAiO3M6MjA6IlVTRV9QUk9EVUNUX1FVQU5USVRZIjtzOjE6IlkiO3M6MjU6IlBST0RVQ1RfUVVBTlRJVFlfVkFSSUFCTEUiO3M6ODoicXVhbnRpdHkiO3M6MjQ6IkFERF9QUk9QRVJUSUVTX1RPX0JBU0tFVCI7czoxOiJZIjtzOjIyOiJQUk9EVUNUX1BST1BTX1ZBUklBQkxFIjtzOjQ6InByb3AiO3M6MjY6IlBBUlRJQUxfUFJPRFVDVF9QUk9QRVJUSUVTIjtzOjE6IlkiO3M6MTg6IlBST0RVQ1RfUFJPUEVSVElFUyI7YTo0OntpOjA7czoxMDoiQVRSX0FVVEhPUiI7aToxO3M6MTI6IkFUUl9MQU5HVUFHRSI7aToyO3M6NzoiQVRSX0FHRSI7aTozO3M6MTA6IkFUUl9PUklHSU4iO31zOjIyOiJPRkZFUlNfQ0FSVF9QUk9QRVJUSUVTIjthOjE6e2k6MDtzOjY6Ik5PVElDRSI7fXM6MjA6IkFERF9UT19CQVNLRVRfQUNUSU9OIjtOO3M6MTU6IkRJU1BMQVlfQ09NUEFSRSI7czoxOiJOIjtzOjE0OiJQQUdFUl9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxNzoiRElTUExBWV9UT1BfUEFHRVIiO3M6MToiWSI7czoyMDoiRElTUExBWV9CT1RUT01fUEFHRVIiO3M6MToiWSI7czoxMToiUEFHRVJfVElUTEUiO3M6MTI6ItCi0L7QstCw0YDRiyI7czoxNzoiUEFHRVJfU0hPV19BTFdBWVMiO3M6MToiTiI7czoyMDoiUEFHRVJfREVTQ19OVU1CRVJJTkciO3M6MToiTiI7czozMToiUEFHRVJfREVTQ19OVU1CRVJJTkdfQ0FDSEVfVElNRSI7czo1OiIzNjAwMCI7czoxNDoiUEFHRVJfU0hPV19BTEwiO3M6MToiTiI7czoyMjoiUEFHRVJfQkFTRV9MSU5LX0VOQUJMRSI7czoxOiJOIjtzOjE1OiJQQUdFUl9CQVNFX0xJTksiO047czoxNzoiUEFHRVJfUEFSQU1TX05BTUUiO047czo5OiJMQVpZX0xPQUQiO3M6MToiWSI7czoxODoiTUVTU19CVE5fTEFaWV9MT0FEIjtzOjIzOiLQn9C+0LrQsNC30LDRgtGMINC10YnRkSI7czoxNDoiTE9BRF9PTl9TQ1JPTEwiO3M6MToiTiI7czoxNDoiU0VUX1NUQVRVU180MDQiO3M6MToiWSI7czo4OiJTSE9XXzQwNCI7czoxOiJZIjtzOjg6IkZJTEVfNDA0IjtzOjg6Ii80MDQucGhwIjtzOjExOiJNRVNTQUdFXzQwNCI7czowOiIiO3M6MTU6IkNPTVBBVElCTEVfTU9ERSI7czoxOiJZIjtzOjI4OiJESVNBQkxFX0lOSVRfSlNfSU5fQ09NUE9ORU5UIjtzOjE6Ik4iO3M6MjI6IlVTRV9FTkhBTkNFRF9FQ09NTUVSQ0UiO3M6MToiWSI7czoxMjoiRU5MQVJHRV9QUk9QIjtzOjA6IiI7czoxMjoiQ09NUEFSRV9QQVRIIjtzOjUwOiIvY2F0YWxvZy1wcm9kdWN0cy9jb21wYXJlLnBocD9hY3Rpb249I0FDVElPTl9DT0RFIyI7czoxNjoiTUVTU19CVE5fQ09NUEFSRSI7czoxNjoi0KHRgNCw0LLQvdC40YLRjCI7czoxMjoiQ09NUEFSRV9OQU1FIjtzOjIwOiJDQVRBTE9HX0NPTVBBUkVfTElTVCI7czoyMjoiTUVTU19TSE9XX01BWF9RVUFOVElUWSI7czoxNDoi0J3QsNC70LjRh9C40LUiO3M6MjQ6IlJFTEFUSVZFX1FVQU5USVRZX0ZBQ1RPUiI7czoyOiIxMCI7czoyNzoiTUVTU19SRUxBVElWRV9RVUFOVElUWV9NQU5ZIjtzOjEwOiLQvNC90L7Qs9C+IjtzOjI2OiJNRVNTX1JFTEFUSVZFX1FVQU5USVRZX0ZFVyI7czo4OiLQvNCw0LvQviI7czoxNzoiUVVBTlRJVFlfSU5fU1RPQ0siO3M6MTc6ItCSINC90LDQu9C40YfQuNC4IjtzOjE4OiJRVUFOVElUWV9PVVRfU1RPQ0siO3M6MTc6ItCf0L7QtCDQt9Cw0LrQsNC3IjtzOjI0OiJISURFX1NFQ1RJT05fREVTQ1JJUFRJT04iO3M6MToiWSI7czoxMjoiVEhJU19VTklDX0lEIjtzOjk6IjFfc2VjdGlvbiI7czo2OiJSRUdJT04iO3M6MDoiIjtzOjIwOiJCWFJfQUpBWF9SRUdJT05fSU5GTyI7czowOiIiO3M6Mjg6IkJYUkVBRFlfRUxFTUVOVF9BRERDTEFTU19CSUciO3M6MDoiIjtzOjMwOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU01BTEwiO3M6MDoiIjtzOjMzOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU1RBTkRBUlQiO3M6MDoiIjtzOjI0OiJCWFJFQURZX0VMRU1FTlRfRFJBV19CSUciO3M6MTk6ImVjb21tZXJjZS5tMi5iaWcudjEiO3M6MjY6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NNQUxMIjtzOjIxOiJlY29tbWVyY2UubTIuc21hbGwudjEiO3M6Mjk6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NUQU5EQVJUIjtzOjE1OiJlY29tbWVyY2UubTIudjEiO3M6MzA6IkJYUkVBRFlfRUxFTUVOVF9FWFRfUEFSQU1TX0JJRyI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjMyOiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TTUFMTCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjM1OiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TVEFOREFSVCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjI4OiJCWFJFQURZX0xJU1RfTUFSS0VSX1RZUEVfQklHIjtzOjE1OiJyaWJib24udmVydGljYWwiO3M6MzA6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TTUFMTCI7czozOiJub3QiO3M6MzM6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TVEFOREFSVCI7czoxNToicmliYm9uLnZlcnRpY2FsIjtzOjMzOiJCWFJFQURZX0xJU1RfT1dOX01BUktFUl9VU0VfU01BTEwiO3M6MToiTiI7czozNjoiQlhSRUFEWV9MSVNUX09XTl9NQVJLRVJfVVNFX1NUQU5EQVJUIjtzOjE6Ik4iO3M6MjI6IkJYUkVBRFlfVVNFUl9UWVBFU19CSUciO3M6MToiTiI7czoyNDoiQlhSRUFEWV9VU0VSX1RZUEVTX1NNQUxMIjtzOjE6Ik4iO3M6Mjc6IkJYUkVBRFlfVVNFUl9UWVBFU19TVEFOREFSVCI7czoxOiJOIjtzOjI4OiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfQklHIjtzOjE6IlkiO3M6MzA6IkJYUkVBRFlfVVNFX0VMRU1FTlRDTEFTU19TTUFMTCI7czoxOiJZIjtzOjMzOiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfU1RBTkRBUlQiO3M6MToiWSI7czoyNjoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9CSUciO3M6MToiWSI7czoyODoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9TTUFMTCI7czoxOiJZIjtzOjMxOiJCWFJFQURZX1ZFUlRJQ0FMX0FMSUdOX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjI6IkJYUl9JTUdfTUFYX0hFSUdIVF9CSUciO3M6MzoiMTgwIjtzOjI0OiJCWFJfSU1HX01BWF9IRUlHSFRfU01BTEwiO3M6MjoiOTAiO3M6Mjc6IkJYUl9JTUdfTUFYX0hFSUdIVF9TVEFOREFSVCI7czozOiIxODAiO3M6MjE6IkJYUl9JTUdfTUFYX1dJRFRIX0JJRyI7czozOiIxODAiO3M6MjM6IkJYUl9JTUdfTUFYX1dJRFRIX1NNQUxMIjtzOjI6IjkwIjtzOjI2OiJCWFJfSU1HX01BWF9XSURUSF9TVEFOREFSVCI7czozOiIxODAiO3M6Mjg6IkJYUl9QUk9EVUNUX0JMT0NLU19PUkRFUl9CSUciO3M6NTE6InBpY3R1cmUscmF0aW5nLG5hbWUsYXJ0aWNsZSxwcmV2aWV3dGV4dCxhY3Rpb250aW1lciI7czozMzoiQlhSX1BST0RVQ1RfQkxPQ0tTX09SREVSX1NUQU5EQVJUIjtzOjM5OiJwaWN0dXJlLG5hbWUsYWN0aW9udGltZXIsYXJ0aWNsZSxyYXRpbmciO3M6MjU6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9CSUciO3M6MToiTiI7czoyNzoiQlhSX1NIT1dfQUNUSU9OX1RJTUVSX1NNQUxMIjtzOjE6Ik4iO3M6MzA6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9TVEFOREFSVCI7czoxOiJOIjtzOjIwOiJCWFJfU0hPV19BUlRJQ0xFX0JJRyI7czoxOiJZIjtzOjIyOiJCWFJfU0hPV19BUlRJQ0xFX1NNQUxMIjtzOjE6IlkiO3M6MjU6IkJYUl9TSE9XX0FSVElDTEVfU1RBTkRBUlQiO3M6MToiWSI7czoyNToiQlhSX1NIT1dfUFJFVklFV19URVhUX0JJRyI7czoxOiJZIjtzOjE5OiJCWFJfU0hPV19SQVRJTkdfQklHIjtzOjM6ImF2ZyI7czoyMToiQlhSX1NIT1dfUkFUSU5HX1NNQUxMIjtzOjM6ImF2ZyI7czoyNDoiQlhSX1NIT1dfUkFUSU5HX1NUQU5EQVJUIjtzOjM6ImF2ZyI7czoxOToiQlhSX1NIT1dfU0xJREVSX0JJRyI7czoxOiJOIjtzOjIxOiJCWFJfU0hPV19TTElERVJfU01BTEwiO3M6MToiWSI7czoyNDoiQlhSX1NIT1dfU0xJREVSX1NUQU5EQVJUIjtzOjE6Ik4iO3M6Mjc6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX0JJRyI7czo2OiJzcXVhcmUiO3M6Mjk6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX1NNQUxMIjtzOjY6InNxdWFyZSI7czozMjoiQlhSX1NLVV9QUk9QU19TSE9XX1RZUEVfU1RBTkRBUlQiO3M6Njoic3F1YXJlIjtzOjIzOiJCWFJfU0xJREVSX0lOVEVSVkFMX0JJRyI7czo0OiIzMDAwIjtzOjI4OiJCWFJfU0xJREVSX0lOVEVSVkFMX1NUQU5EQVJUIjtzOjQ6IjMwMDAiO3M6Mjg6IkJYUl9TTElERVJfUFJPR1JFU1NfU1RBTkRBUlQiO3M6MToiTiI7czoyODoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX0JJRyI7czoxOiJZIjtzOjMwOiJCWFJfVElMRV9TSE9XX1BST1BFUlRJRVNfU01BTEwiO3M6MToiTiI7czozMzoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjE6IkJYUl9VU0VfRkFTVF9WSUVXX0JJRyI7czoxOiJOIjtzOjIzOiJCWFJfVVNFX0ZBU1RfVklFV19TTUFMTCI7czoxOiJOIjtzOjI2OiJCWFJfVVNFX0ZBU1RfVklFV19TVEFOREFSVCI7czoxOiJOIjtzOjIyOiJNRVNTX0JUTl9GQVNUX1ZJRVdfQklHIjtzOjMxOiLQkdGL0YHRgtGA0YvQuSDQv9GA0L7RgdC80L7RgtGAIjtzOjI0OiJNRVNTX0JUTl9GQVNUX1ZJRVdfU01BTEwiO3M6MzE6ItCR0YvRgdGC0YDRi9C5INC/0YDQvtGB0LzQvtGC0YAiO3M6Mjc6Ik1FU1NfQlROX0ZBU1RfVklFV19TVEFOREFSVCI7czozMToi0JHRi9GB0YLRgNGL0Lkg0L/RgNC+0YHQvNC+0YLRgCI7czoyMDoiUFJPRFVDVF9ST1dfVkFSSUFOVFMiO3M6MTY2OiJbeydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX1dIjtzOjEzOiJQUk9QRVJUWV9DT0RFIjthOjI6e2k6MDtzOjg6IkFUUl9JU0JOIjtpOjE7czoxMzoiQVRSX1BVQkxJU0hFUiI7fXM6MTc6IkNVUlJFTlRfQkFTRV9QQUdFIjtzOjU1OiIvY2F0YWxvZy1wcm9kdWN0cy80NTc3Lz9zb3J0PVBST1BFUlRZX1JBVElORyZvcmRlcj1kZXNjIjtzOjExOiJQQVJFTlRfTkFNRSI7czoyMzoiYnhyZWFkeS5tYXJrZXQyOmNhdGFsb2ciO3M6MjA6IlBBUkVOVF9URU1QTEFURV9OQU1FIjtzOjEzOiJyZWxvZC1jYXRhbG9nIjtzOjIwOiJQQVJFTlRfVEVNUExBVEVfUEFHRSI7czo3OiJzZWN0aW9uIjtzOjEzOiJHTE9CQUxfRklMVEVSIjthOjE6e3M6MTg6IiFQUk9QRVJUWV9BVFJfSVNCTiI7czo1OiJnaWZ0JSI7fX0=.7362603a48911c14b147aefd65ad1b03dae6fef1c4f18d7a767e06a5ab56cdf6",
    "action":	"showMore",
    "PAGEN_1": "1"
}

В консоли браузера (фильтр на XHR) заметил, что при  нажатии кнопки "показать еще" на сайте для подгрузки очередной страницы с карточками книг, делается post запрос на адрес `https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php` с данными `data`, который подгружает дополнительную страницу сайта c карточками книг. Фактически происходит обращение к скрытому API сайта и джаваскрипт выгружает данные на страницу. Поэтому получать страницы можно либо варьируя параметр `PAGEN_1` в урле, либо через обращение напрямую к ajax.php. Сравним время работы

In [6]:
n_page_first = 1
n_page_last = 90

def get_site_page(page_id, ajax=True):
    if ajax:
        data['PAGEN_1'] = str(page_id)
        page = post_page('https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php', data=data, headers={'user-agent': UA})
    else:
        page = get_page(f"https://shop.relod.ru/catalog-products/4577/?sort=PROPERTY_RATING&order=desc&PAGEN_1={page_id}", headers={'user-agent': UA})
    return page
#with ThreadPool(30) as pool:
#pages = map(get_site_page, range(n_page_first, n_page_last+1))
#pool.join()

In [7]:
def get_card_urls(pages):
    card_urls = []
    for i, page in tqdm(enumerate(pages, 1)):
        if page is None:
            print(f"Страница {i} не скачана", file=sys.stderr)
            continue
        soup = BeautifulSoup(page.text, 'lxml')
        card_urls.extend(map(lambda s: DOMAIN + s.find('a').attrs['href'], soup.find_all('div', class_='bxr-element-image'))) 
    return card_urls

In [8]:
%%time
pages = map(partial(get_site_page, ajax=True), range(n_page_first, n_page_last+1))
pages = [page for page in pages]

CPU times: user 2.95 s, sys: 205 ms, total: 3.16 s
Wall time: 1min 37s


In [9]:
%%time
pages = map(partial(get_site_page, ajax=False), range(n_page_first, n_page_last+1))
pages = [page for page in pages]

CPU times: user 2.99 s, sys: 216 ms, total: 3.21 s
Wall time: 1min 13s


Вариант без ajax быстрее

Добавим многопоточность

In [10]:
%%time
with ThreadPool(10) as pool:
    pages = pool.map(partial(get_site_page, ajax=False), range(n_page_first, n_page_last+1))
pool.join()

CPU times: user 2.11 s, sys: 111 ms, total: 2.22 s
Wall time: 8.52 s


In [11]:
card_urls = get_card_urls(pages)
card_urls[:10]

0it [00:00, ?it/s]

['https://shop.relod.ru/catalog-products/winter_s_tale_movie_tie_in_international_edition/',
 'https://shop.relod.ru/catalog-products/white-death/',
 'https://shop.relod.ru/catalog-products/what_ho/',
 'https://shop.relod.ru/catalog-products/warlock/',
 'https://shop.relod.ru/catalog-products/war_and_peace_9780199232765/',
 'https://shop.relod.ru/catalog-products/walk-the-wire/',
 'https://shop.relod.ru/catalog-products/voyage-au-centre-de-la-terre-pocket-classiques/',
 'https://shop.relod.ru/catalog-products/very_good_jeeves/',
 'https://shop.relod.ru/catalog-products/vernon_god_little_audiobook_on_cds/',
 'https://shop.relod.ru/catalog-products/vanity-fair/']

In [12]:
card_urls = list(OrderedDict.fromkeys(card_urls,0).keys())

In [13]:
card_urls[:10]

['https://shop.relod.ru/catalog-products/winter_s_tale_movie_tie_in_international_edition/',
 'https://shop.relod.ru/catalog-products/white-death/',
 'https://shop.relod.ru/catalog-products/what_ho/',
 'https://shop.relod.ru/catalog-products/warlock/',
 'https://shop.relod.ru/catalog-products/war_and_peace_9780199232765/',
 'https://shop.relod.ru/catalog-products/walk-the-wire/',
 'https://shop.relod.ru/catalog-products/voyage-au-centre-de-la-terre-pocket-classiques/',
 'https://shop.relod.ru/catalog-products/very_good_jeeves/',
 'https://shop.relod.ru/catalog-products/vernon_god_little_audiobook_on_cds/',
 'https://shop.relod.ru/catalog-products/vanity-fair/']

In [14]:
class GetPageError(RuntimeError):
    pass

In [15]:
def process_page(url):
    record = {}
    record['url'] = url
    page = get_page(url)
    if not page:
        raise GetPageError(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    record['Название'] = soup.find('h1', itemprop='name').text

    record['Иллюстрации'] = [DOMAIN + s.attrs['data-src'] for s in soup.find('div',class_="bxr-element-slider-main").find_all('img', itemprop="image")]

    record['Метки'] = soup.find('div', class_='bxr-ribbon-marker-vertical').text.split()

    # Если вместо itemprop задать class_='bxr-raiting-detail' то для нескольких десятков книг работать не будет, так как рейтиг просто не задан
    # тегов meta нету, поэтому будем брать attrs у None объекта
    # маркером таких карточек - отсутствие 'aggregateRating'
    elements = soup.find('div', itemprop='aggregateRating')
    if elements:
        record['Оценка'] = float(elements.find('meta', itemprop='ratingValue').attrs['content']) 
        rating_count = elements.find('meta', itemprop='ratingCount').attrs['content']
        record['Число голосов'] = rating_count if rating_count else 0
    else:
        record['Оценка'], record['Число голосов'] = 0, 0

    record['Наличие'] = soup.find('div', itemprop='availability').text

    record['Цена'] = float(soup.find('meta', itemprop='price').attrs['content'])

    search_res = re.search(r'[\"\']PRICE[\"\']\s*:\s*(\d+(\.\d+)?)', page.text, flags=re.DOTALL)
    if search_res is not None:
        record['Цена (скидка)'] = float(search_res.group(1))

    record['Описание'] = re.sub(r'\s+', ' ', soup.find('div', class_='bxr-detail-tab-content').text).strip()

    rows = soup.find('table', class_='bxr-props-table').find_all('tr')
    for row in rows:
        record[row.find('td', class_='bxr-props-name').text] = row.find('td', class_='bxr-props-data').find(itemprop='value').text.strip()

    return record

In [16]:
#https://shop.relod.ru/catalog-products/warlock/
#"https://shop.relod.ru/catalog-products/thrilogy-book-1-wolf-hall/"
# https://shop.relod.ru/catalog-products/trouver-le-chemin/

In [17]:
record = process_page("https://shop.relod.ru/catalog-products/warlock/")
record

{'url': 'https://shop.relod.ru/catalog-products/warlock/',
 'Название': 'Warlock',
 'Иллюстрации': ['https://shop.relod.ru/upload/iblock/958/95898795761e64aec5907a306abec79e.jpg'],
 'Метки': ['Sale', 'ПТВ'],
 'Оценка': 0.0,
 'Число голосов': 0,
 'Наличие': 'В наличии',
 'Цена': 232.0,
 'Цена (скидка)': 185.6,
 'Описание': 'After the death of his beloved Queen Lostris, Taita performs the rites of embalmment and burial for her. Then, stricken with grief, he retreats into the forbidding deserts of North Africa where he becomes a hermit. Over the years that follow he devotes himself to the study of the mysteries of the occult until, armed with these extraordinary powers, he gradually transforms himself into the Warlock. When he is called upon to save the dynasty of Queen Lostris from the clouds of evil that loom over Egypt he discovers that there has, after all, been a divine purpose in his bereavement and suffering ...',
 'ISBN(EAN)': '9780330376303',
 'Издатель': 'Pan Macmillan',
 'Автор

In [18]:
print(json.dumps(record, ensure_ascii=False, indent=1))

{
 "url": "https://shop.relod.ru/catalog-products/warlock/",
 "Название": "Warlock",
 "Иллюстрации": [
  "https://shop.relod.ru/upload/iblock/958/95898795761e64aec5907a306abec79e.jpg"
 ],
 "Метки": [
  "Sale",
  "ПТВ"
 ],
 "Оценка": 0.0,
 "Число голосов": 0,
 "Наличие": "В наличии",
 "Цена": 232.0,
 "Цена (скидка)": 185.6,
 "Описание": "After the death of his beloved Queen Lostris, Taita performs the rites of embalmment and burial for her. Then, stricken with grief, he retreats into the forbidding deserts of North Africa where he becomes a hermit. Over the years that follow he devotes himself to the study of the mysteries of the occult until, armed with these extraordinary powers, he gradually transforms himself into the Warlock. When he is called upon to save the dynasty of Queen Lostris from the clouds of evil that loom over Egypt he discovers that there has, after all, been a divine purpose in his bereavement and suffering ...",
 "ISBN(EAN)": "9780330376303",
 "Издатель": "Pan Macmi

In [19]:
if os.path.isdir('data_hw'):
    shutil.rmtree('data_hw') # УДАЛЯЕМ ДИРЕКТОРИЮ! 
os.mkdir('data_hw') # теперь есть пустая директория

In [20]:
def process_page_wrapper(i):
    with gzip.open('data_hw/part_{:05d}.jsonl.gz'.format(i), mode='wt') as f_json:
        #f_json = codecs.getwriter('utf8')(f_json)
        while not queue.empty():
            try:
                card_url = queue.get()
                record = process_page(card_url)
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
            except KeyboardInterrupt:
                with lock:
                    print("Прерывание процесса обкачки", file=sys.stderr)
                break
            except Exception as e:
                with lock:
                    print(f"Возникло исключение: {e}", file=sys.stderr)
                    print(f"url: {card_url}", file=sys.stderr)
            finally:
                with lock:
                    pbar.update(1)

In [21]:
queue = Queue()
for card_url in card_urls:
    queue.put(card_url)

In [22]:
with ThreadPool(30) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

  0%|          | 0/1777 [00:00<?, ?it/s]

Возникло исключение: HTTPSConnectionPool(host='shop.relod.ru', port=443): Max retries exceeded with url: /catalog-products/the-awakening/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fea6574c550>, 'Connection to shop.relod.ru timed out. (connect timeout=None)'))
url: https://shop.relod.ru/catalog-products/the-awakening/
Возникло исключение: 'NoneType' object has no attribute 'text'
url: https://shop.relod.ru/catalog-products/narrative_of_the_life_of_frederick_douglass/
Возникло исключение: 'NoneType' object has no attribute 'text'
url: https://shop.relod.ru/catalog-products/chance/
Возникло исключение: 'NoneType' object has no attribute 'text'
url: https://shop.relod.ru/catalog-products/battlefield-3-the-russian/
Возникло исключение: 'NoneType' object has no attribute 'text'
url: https://shop.relod.ru/catalog-products/three_to_get_deadly/
Возникло исключение: HTTPSConnectionPool(host='shop.relod.ru', port=443): Max retries exceeded with url: /catalog-

Исключения  'NoneType' object has no attribute 'text', соответсвтуют случаю когда карточки нету в магазине, и сайт выдает просто страницу каталога, которая естественно имеет другую структуру

In [23]:
!ls data_hw

part_00000.jsonl.gz  part_00010.jsonl.gz  part_00020.jsonl.gz
part_00001.jsonl.gz  part_00011.jsonl.gz  part_00021.jsonl.gz
part_00002.jsonl.gz  part_00012.jsonl.gz  part_00022.jsonl.gz
part_00003.jsonl.gz  part_00013.jsonl.gz  part_00023.jsonl.gz
part_00004.jsonl.gz  part_00014.jsonl.gz  part_00024.jsonl.gz
part_00005.jsonl.gz  part_00015.jsonl.gz  part_00025.jsonl.gz
part_00006.jsonl.gz  part_00016.jsonl.gz  part_00026.jsonl.gz
part_00007.jsonl.gz  part_00017.jsonl.gz  part_00027.jsonl.gz
part_00008.jsonl.gz  part_00018.jsonl.gz  part_00028.jsonl.gz
part_00009.jsonl.gz  part_00019.jsonl.gz  part_00029.jsonl.gz


In [24]:
def records_reader(dirname: str):
    with ExitStack() as stack:
        files = [stack.enter_context(gzip.open(os.path.join(dirname, fname), mode='rt')) for fname in os.listdir(dirname)]
        # Все открытые файлы будут автоматически закрыты в 
        # конце оператора `with`, даже если попытки открыть 
        # файлы позже в списке вызовут исключение
        yield from map(json.loads, chain.from_iterable(files))

In [25]:
df = pd.DataFrame(records_reader('data_hw'))

In [26]:
df.shape

(1771, 29)

In [27]:
df.columns

Index(['url', 'Название', 'Иллюстрации', 'Метки', 'Оценка', 'Число голосов',
       'Наличие', 'Цена', 'Описание', 'ISBN(EAN)', 'Издатель', 'Автор', 'Язык',
       'Возрастные ограничения', 'Рекомендуемый возраст', 'Формат', 'Страницы',
       'Год издания', 'Рейтинг', 'Вес (грамм)', 'Размер (мм)', 'Серия',
       'Читательская аудитория', 'Носитель', 'Тип продукта', 'Цена (скидка)'],
      dtype='object')

In [28]:
df.head()

,url,Название,Иллюстрации,Метки,Оценка,Число голосов,Наличие,Цена,Описание,ISBN(EAN),Издатель,Автор,Язык,Серия,Возрастные ограничения,Рекомендуемый возраст,Формат,Страницы,Год издания,Рейтинг,Вес (грамм),Размер (мм),Носитель,Читательская аудитория,Тип продукта,Цена (скидка),Страна происхождения,Издание,Обзор/Ролик
0,https://shop.relod.ru/catalog-products/the_vir...,The Virginian,[https://shop.relod.ru/upload/iblock/c4c/c4c76...,[],0.0,0,Под заказ,1119.0,The Virginian (1902) is Owen Wister's classic ...,9780199554102,Oxford University Press,Oscar Wilde,Английский,Oxford World's Classics,16+,16+,Мягкая обложка,400,2009,4.8,280,197(д) х 129(ш) х 18(в),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://shop.relod.ru/catalog-products/the-sea...,The Searcher,[https://shop.relod.ru/upload/iblock/434/cddzs...,[],0.0,0,В наличии,1258.0,A DISAPPEARANCE. A SMALL TOWN. A QUESTION THAT...,9780241990100,Penguin Books,Tana French,Английский,NaN,16+,16+,Мягкая обложка,400,2022,4.8,277,198(д) х 129(ш) х 24(в),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://shop.relod.ru/catalog-products/the_ice...,The Ice Queen,[https://shop.relod.ru/upload/iblock/be1/be171...,[Sale],0.0,0,Только в магазине,763.0,Some secrets remain buried forever ...Holocaus...,9781447266860,Pan Macmillan,Neuhaus Nele,Английский,NaN,16+,16+,Мягкая обложка,352,2015,4.8,465,235(д) х 152(ш) х 26(в),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://shop.relod.ru/catalog-products/the_com...,The Combat,[https://shop.relod.ru/upload/iblock/acd/acd13...,[Sale],0.0,0,В наличии,243.0,"'The Combat', published in 1972, unforgettably...",9780143185536,Penguin Group,Omotoso Kole,Английский,NaN,16+,16+,Мягкая обложка,124,2008,4.8,143,201(д) х 126(ш) х 11(в),Печатная книга,NaN,NaN,NaN,NaN,NaN,NaN
4,https://shop.relod.ru/catalog-products/rub_iy_...,Rubáiyát of Omar Khayyám,[https://shop.relod.ru/upload/iblock/5b8/5b8bf...,[],0.0,0,В наличии,994.0,"'The Moving Finger writes; and, having writ Mo...",9780199580507,Oxford University Press,FitzGerald Edward,Английский,Oxford World's Classics,16+,16+,Мягкая обложка,240,2010,4.8,182,196(д) х 130(ш) х 14(в),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771 entries, 0 to 1770
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   url                     1771 non-null   object 
 1   Название                1771 non-null   object 
 2   Иллюстрации             1771 non-null   object 
 3   Метки                   1771 non-null   object 
 4   Оценка                  1771 non-null   float64
 5   Число голосов           1771 non-null   object 
 6   Наличие                 1771 non-null   object 
 7   Цена                    1771 non-null   float64
 8   Описание                1771 non-null   object 
 9   ISBN(EAN)               1771 non-null   object 
 10  Издатель                1769 non-null   object 
 11  Автор                   1694 non-null   object 
 12  Язык                    1770 non-null   object 
 13  Серия                   401 non-null    object 
 14  Возрастные ограничения  1704 non-null   

In [30]:
df.describe(include=['object'])

,url,Название,Иллюстрации,Метки,Число голосов,Наличие,Описание,ISBN(EAN),Издатель,Автор,Язык,Серия,Возрастные ограничения,Рекомендуемый возраст,Формат,Страницы,Год издания,Рейтинг,Вес (грамм),Размер (мм),Носитель,Читательская аудитория,Тип продукта,Страна происхождения,Издание,Обзор/Ролик
count,1771,1771,1771,1771,1771,1771,1771,1771,1769,1694,1770,401,1704,1705,1744,1727,1767,1709,1755,1765,118,72,22,4,1,2
unique,1771,1697,1771,6,6,4,1738,1771,109,860,3,12,4,4,4,248,44,15,436,924,5,2,3,3,1,1
top,https://shop.relod.ru/catalog-products/the_vir...,The Hobbit,[https://shop.relod.ru/upload/iblock/c4c/c4c76...,[],0,Под заказ,,9780199554102,Harper Collins,J.R.R. Tolkien,Английский,Oxford World's Classics,16+,16+,Мягкая обложка,400,2008,4.0,200,198(д) х 129(ш) х 20(в),Печатная книга,General (US: Trade),Аудио,Соединенное Королевство,Second Edition,Просмотреть
freq,1,4,1,1312,1743,923,18,1,331,39,1557,227,1696,1696,1639,78,203,606,50,29,101,71,15,2,1,2


In [31]:
df.describe()

,Оценка,Цена,Цена (скидка)
count,1771.000000,1771.000000,58.000000
mean,0.064370,1043.384529,232.055172
std,0.529833,730.778142,174.616306
min,0.000000,99.000000,79.200000
25%,0.000000,490.000000,143.200000
50%,0.000000,1101.000000,185.600000
75%,0.000000,1357.000000,264.000000
max,5.000000,13213.000000,855.200000


In [32]:
df.to_csv('hw_parsing.csv', index=False)